In [1]:
import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
!pip freeze > requirements.txt
# device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cpu'

hidden_size = 10
input_size = 1
output_size = 1
num_layers = 1
lr = 0.01
# number of points
num_time_steps = 50

In [2]:
if not os.path.exists('data/train.pkl'):
    data = pd.read_csv(f'data/LD2011_2014.txt', sep=";")
    data.columns = ['Date'] + list(data.columns[1:])
    data[data.columns[1:]] = data[data.columns[1:]].apply(lambda x: [str(el).replace(',', '.') for el in x]).astype('float64')
    data['Date'] = data['Date'].astype('datetime64[ns]')
    train, validation, test = \
                np.split(data.sample(frac=1, random_state=42), 
                        [int(.7*len(data)), int(.9*len(data))])
                
    pickle.dump(train, open('data/train.pkl', 'wb'))
    pickle.dump(validation, open('data/validation.pkl', 'wb'))
    pickle.dump(test, open('data/test.pkl', 'wb'))
                
    print(f'Original Dataset: {len(data)}\nTrain Split: {len(train)}\nValidation Split: {len(validation)}\nTest Split: {len(test)}')
else:
    train = pickle.load(open('data/train.pkl', 'rb'))
    validation = pickle.load(open('data/validation.pkl', 'rb'))
    test = pickle.load(open('data/test.pkl', 'rb'))
    print(f'Train Split: {len(train)}\nValidation Split: {len(validation)}\nTest Split: {len(test)}')
    

Train Split: 98179
Validation Split: 28051
Test Split: 14026


In [3]:
class Net(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super(Net, self).__init__(*args, **kwargs)
        
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            device=device
        )
        
        self.linear = nn.Linear(hidden_size, output_size, device=device)
        
    def forward(self, x, hidden_prev):
        out, hidden_prev = self.rnn(x, hidden_prev)
        out = out.reshape(-1, hidden_size)
        
        out = self.linear(out)
        out = out.unsqueeze(dim=0)
        return out, hidden_prev